In [18]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import geopandas as gpd
import json

In [13]:
# csv 파일로 저장된 raw 데이터 df로 불러옴 
df = pd.read_csv('raw/서울시_상권분석서비스(영역-상권).csv', encoding='utf-8')

In [14]:
# raw 데이터 확인
df.head(10)

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872
5,A,골목상권,3110019,창덕궁,199222,452926,11110,종로구,11110615,"종로1,2,3,4가동",40999
6,A,골목상권,3110020,서울국제고등학교,199598,454302,11110,종로구,11110650,혜화동,133692
7,A,골목상권,3110001,이북5도청사,196262,456660,11110,종로구,11110560,평창동,108529
8,A,골목상권,3110002,독립문역 1번,196220,452912,11110,종로구,11110570,무악동,31531
9,A,골목상권,3110003,세검정초등학교,196389,455998,11110,종로구,11110550,부암동,191570


In [15]:
# 바꾸고자 하는 x, y좌표가 기록된 칼럼 지정
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['엑스좌표_값'], df['와이좌표_값']))

# 현재 좌표계 설정
gdf.crs = 'epsg:5181'

# 좌표계 변환
gdf_4162 = gdf.to_crs('epsg:4162')

# 변환 좌표 속성 생성 
gdf['경도'] = gdf_4162['geometry'].x
gdf['위도'] = gdf_4162['geometry'].y

In [16]:
gdf.head(5)

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,geometry,경도,위도
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264,POINT (197093.000 453418.000),126.969189,37.577515
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306,POINT (196991.000 455057.000),126.968028,37.592283
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415,POINT (197064.000 456643.000),126.968848,37.606575
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855,POINT (198581.000 453781.000),126.986036,37.580789
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872,POINT (198883.000 453690.000),126.989455,37.579969


In [28]:
'''
각각의 칼럼 데이터를 numpy array로 만들어서 문자열 데이터는 그대로 리스트에 저장,
숫자 데이터는 각각의 데이터 타입으로 변환시켜 리스트에 저장 (numpy array로 생성된 숫자 배열은 int64이기 때문에 변환 필요),
zip으로 묶은 뒤 반복문으로 양식에 입력

'''
geopoint = {
    "type": "FeatureCollection",
	"name": "haengpoint",
	"features": [
        {
			"type": "Feature",
			"geometry": {
				"type": "Point",
				"coordinates": [lon, lat]
			},
			"properties": {
				"cd_div_cd": sgc,
				"cd_div_nm": sgcn,
				"cd_cd": sc,
                "cd_nm": scn,
                "br_cd": jc,
                "br_nm": jcn,
                "ab_cd": hc,
                "ab_nm": hcn
			}
		} for sgc, sgcn, sc, scn, jc, jcn, hc, hcn, lat, lon in zip(list(np.array(df['상권_구분_코드'])),
                                                              list(np.array(df['상권_구분_코드_명'])),
                                                              list(map(int, np.array(df['상권_코드']))),
                                                              list(np.array(df['상권_코드_명'])),
                                                              list(map(int, np.array(df['자치구_코드']))),
                                                              list(np.array(df['자치구_코드_명'])),
                                                              list(map(int, np.array(df['행정동_코드']))),
                                                              list(np.array(df['행정동_코드_명'])),
                                                              list(map(float, np.array(df['위도']))),
                                                              list(map(float, np.array(df['경도']))))
    ]
}

In [30]:
# 생성된 geopoint json 형식 확인
geopoint

{'type': 'FeatureCollection',
 'name': 'haengpoint',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [126.96918907282516, 37.577514546904446]},
   'properties': {'cd_div_cd': 'A',
    'cd_div_nm': '골목상권',
    'cd_cd': 3110008,
    'cd_nm': '배화여자대학교(박노수미술관)',
    'br_cd': 11110,
    'br_nm': '종로구',
    'ab_cd': 11110515,
    'ab_nm': '청운효자동'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [126.9680276291576, 37.59228320443988]},
   'properties': {'cd_div_cd': 'A',
    'cd_div_nm': '골목상권',
    'cd_cd': 3110009,
    'cd_nm': '자하문터널',
    'br_cd': 11110,
    'br_nm': '종로구',
    'ab_cd': 11110550,
    'ab_nm': '부암동'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [126.96884808672156, 37.60657479690501]},
   'properties': {'cd_div_cd': 'A',
    'cd_div_nm': '골목상권',
    'cd_cd': 3110010,
    'cd_nm': '평창동서측',
    'br_cd': 11110,
    'br_nm': '종로구',
    'ab_cd': 11110560,
    'ab_nm': '평창동'}},
  {'t

In [31]:
# json 파일로 output 폴더에 저장
with open('output/geopoint.json', 'w') as f:
    json.dump(geopoint, f, indent=4, ensure_ascii=False)

In [19]:
# 변환 후 불필요한 칼럼 제거
gdf = gdf.drop(columns=['geometry','엑스좌표_값','와이좌표_값'])
gdf

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,경도,위도
0,A,골목상권,3110008,배화여자대학교(박노수미술관),11110,종로구,11110515,청운효자동,149264,126.969189,37.577515
1,A,골목상권,3110009,자하문터널,11110,종로구,11110550,부암동,178306,126.968028,37.592283
2,A,골목상권,3110010,평창동서측,11110,종로구,11110560,평창동,369415,126.968848,37.606575
3,A,골목상권,3110017,정독도서관,11110,종로구,11110600,가회동,83855,126.986036,37.580789
4,A,골목상권,3110018,중앙고등학교,11110,종로구,11110600,가회동,166872,126.989455,37.579969
...,...,...,...,...,...,...,...,...,...,...,...
1645,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,11140,중구,11140520,소공동,983618,126.983957,37.561262
1646,U,관광특구,3001493,동대문패션타운 관광특구,11140,중구,11140590,광희동,606058,127.013377,37.564524
1647,U,관광특구,3001494,"종로,청계 관광특구",11110,종로구,11110615,"종로1,2,3,4가동",653127,126.999793,37.567210
1648,U,관광특구,3001495,잠실 관광특구,11710,송파구,11710562,방이2동,2462734,127.117353,37.513628


In [22]:
# csv 파일로 저장
gdf.to_csv('raw/서울시_상권분석서비스(영역-상권)_epsg_4162.csv', encoding='utf-8')